In [17]:
# Conectar Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# Librerías necesarias
import sqlite3
import pandas as pd
from pathlib import Path

# Cargar los datos
def load_data(db_path):
    db_path = Path(db_path)
    conn = sqlite3.connect(db_path)
    df_movies = pd.read_sql_query("SELECT * FROM movies", conn)
    df_ratings = pd.read_sql_query("SELECT * FROM ratings", conn)
    conn.close()
    df = pd.merge(df_ratings, df_movies, on='movieId', how='left')
    return df

Extraer año de title


In [21]:
import re

def extract_year(df):
    df['year'] = df['title'].str.extract(r'\((\d{4})\)').astype(float)
    return df


In [22]:
df = load_data('/content/drive/MyDrive/bd_movies_recommendation/data/db_movies.sqlite')

Convertir genres en columnas dummy

In [23]:
def process_genres(df):
    df['genres_list'] = df['genres'].str.split('|')
    df_genres = df['genres_list'].str.join('|').str.get_dummies(sep='|')
    df = pd.concat([df, df_genres], axis=1)
    return df


Agregar estadísticas por película y usuario

In [24]:
def add_statistics(df):
    # Promedio por película
    movie_avg = df.groupby('movieId')['rating'].mean().rename('movie_avg_rating')
    df = df.merge(movie_avg, on='movieId', how='left')

    # Promedio por usuario
    user_avg = df.groupby('userId')['rating'].mean().rename('user_avg_rating')
    df = df.merge(user_avg, on='userId', how='left')

    return df


Aplicar funciones al DataFrame combinado (df)

In [25]:
df = extract_year(df)
df = process_genres(df)
df = add_statistics(df)


In [26]:
df.head()
print(df.columns)


Index(['userId', 'movieId', 'rating', 'timestamp', 'title', 'genres', 'year',
       'genres_list', '(no genres listed)', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western', 'movie_avg_rating',
       'user_avg_rating'],
      dtype='object')
